In [1]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms import *
from models import *
from dataloaders import *

from tabulate import tabulate

In [2]:
device = 'cpu'
dataset = AdultDataset(device=device)

In [3]:
def lr_kde_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = MLP(num_features=dataset.XZ_train.shape[1],n_layers=[4])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=False, 
            delta_effort=hp['delta_effort'],
            effort_iter=hp['effort_iter'],
            effort_lr=hp['effort_lr']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds):
    
    _, _, SGD = lr_kde_model_runner(dataset, SGD_hp, seeds)
    _, _, EI = lr_kde_model_runner(dataset, EI_hp, seeds)
    _, _, DP = lr_kde_model_runner(dataset, DP_hp, seeds)
    _, _, EO = lr_kde_model_runner(dataset, EO_hp, seeds)
    _, _, EODD = lr_kde_model_runner(dataset, EODD_hp, seeds)
    
    return SGD, EI, DP, EO, EODD

def hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_kde_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

#### SGD Hyperparameter Selection

In [5]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 100
SGD_hp_test['batch_size'] = 1024 
SGD_hp_test['fairness'] = ''
SGD_hp_test['h'] = 0.01
SGD_hp_test['delta_huber'] = 0.5
SGD_hp_test['delta_effort'] = 2.1
SGD_hp_test['effort_iter'] = 20
SGD_hp_test['effort_lr'] = 15

hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:24<00:00,  4.11epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:25<00:00,  3.88epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:26<00:00,  3.72epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:24<00:00,  4.13epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ---------  ---------
         0.0001          0          0.789962        0.797945  0          0.0562983  0.0608535  0.0608535
         0.001           0          0.850429        0.850157  0.0391528  0.124529   0.0940278  0.0940278
         0.01            0          0.851506        0.855296  0.0546629  0.127716   0.0998797  0.0998797
         0.1             0          0.846906        0.845185  0.0438769  0.11466    0.141118   0.141118


The learning rate is decided as 0.01.


In [6]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.01
SGD_hp['lambda_'] = 0


#### EI Hyperparameter Test

In [7]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.01]
EI_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EI_hp_test['fairness'] = 'EI'

hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:50<00:00,  1.97epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:47<00:00,  2.12epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:47<00:00,  2.12epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:46<00:00,  2.15epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei         dp          eo       eodd
---------------  ---------  ----------------  --------------  ----------  ---------  ----------  ---------
           0.01       0             0.851506        0.855296  0.0546629   0.127716   0.0998797   0.0998797
           0.01       0.2           0.851341        0.85629   0.030667    0.111953   0.106496    0.106496
           0.01       0.6           0.848357        0.851649  0.0263245   0.0980282  0.0475695   0.0475695
           0.01       0.8           0.843466        0.848168  0.0106989   0.0800713  0.011561    0.0162681
           0.01       0.9           0.845083        0.8485    0.0116066   0.0843431  0.00747288  0.0181995
           0.01       0.95          0.823573        0.824134  0.00486729  0.0476022  0.0856374   0.0856374


The lambda for EI is decided as 0.9.

In [8]:
EI_hp = EI_hp_test.copy()
EI_hp['learning_rate'] = 0.01
EI_hp['lambda_'] = 0.9

#### DP Hyperparameter Selection

In [9]:
DP_hp_test = SGD_hp_test.copy()
DP_hp_test['learning_rate'] = [0.01]
DP_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
DP_hp_test['fairness'] = 'DP'

hyperparameter_test(dataset, DP_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:24<00:00,  4.00epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:25<00:00,  3.93epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:25<00:00,  3.92epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:25<00:00,  3.91epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei          dp         eo       eodd
---------------  ---------  ----------------  --------------  ----------  ----------  ---------  ---------
           0.01       0             0.851506        0.855296  0.0546629   0.127716    0.0998797  0.0998797
           0.01       0.2           0.852252        0.85397   0.0486469   0.108452    0.0645543  0.0645543
           0.01       0.6           0.847859        0.849329  0.0593721   0.0688897   0.0664709  0.0664709
           0.01       0.8           0.825148        0.832587  0.0141283   0.0250389   0.180257   0.180257
           0.01       0.9           0.817315        0.818333  0.00491643  0.0104512   0.23587    0.23587
           0.01       0.95          0.810726        0.816841  0.00220036  0.00752429  0.227102   0.227102


The lambda for DP is decided as 0.8.

In [10]:
DP_hp = DP_hp_test.copy()
DP_hp['learning_rate'] = 0.01
DP_hp['lambda_'] = 0.8

#### EO Hyperparameter Selection

In [11]:
EO_hp_test = SGD_hp_test.copy()
EO_hp_test['learning_rate'] = [0.01]
EO_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EO_hp_test['fairness'] = 'EO'

hyperparameter_test(dataset, EO_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:25<00:00,  3.93epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:25<00:00,  3.87epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:26<00:00,  3.83epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:25<00:00,  3.91epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp          eo       eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ----------  ---------
           0.01       0             0.851506        0.855296  0.0546629  0.127716   0.0998797   0.0998797
           0.01       0.2           0.851382        0.855793  0.0605017  0.115099   0.059885    0.059885
           0.01       0.6           0.849269        0.849494  0.0432451  0.0890204  0.00363959  0.0214757
           0.01       0.8           0.850388        0.854964  0.0392648  0.115375   0.0416463   0.0416463
           0.01       0.9           0.845414        0.84303   0.0247419  0.106425   0.0487521   0.0487521
           0.01       0.95          0.749513        0.757335  0          0          0           0


The lambda for EO is decided as 0.6.

In [12]:
EO_hp = EO_hp_test.copy()
EO_hp['learning_rate'] = 0.01
EO_hp['lambda_'] = 0.6

#### EODD Hyperparameter Selection

In [13]:
EODD_hp_test = SGD_hp_test.copy()
EODD_hp_test['learning_rate'] = [0.01]
EODD_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EODD_hp_test['fairness'] = 'EODD'

hyperparameter_test(dataset, EODD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:27<00:00,  3.66epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:27<00:00,  3.67epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:27<00:00,  3.69epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:27<00:00,  3.68epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei        dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  --------  ---------  ---------
           0.01       0             0.851506        0.855296  0.0546629  0.127716  0.0998797  0.0998797
           0.01       0.2           0.851175        0.852975  0.0636406  0.120421  0.043308   0.043308
           0.01       0.6           0.851838        0.854467  0.0528204  0.100624  0.0309008  0.0309008
           0.01       0.8           0.848523        0.851649  0.048324   0.113423  0.0286681  0.0379719
           0.01       0.9           0.749513        0.757335  0          0         0          0
           0.01       0.95          0.749513        0.757335  0          0         0          0


The lambda for EODD is decided as 0.6.

In [15]:
EODD_hp = EODD_hp_test.copy()
EODD_hp['learning_rate'] = 0.01
EODD_hp['lambda_'] = 0.6

#### Model training

In [16]:
seeds = np.arange(1,6)

SGD, EI, DP, EO, EODD = experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds)

/var/folders/mt/mgpv1mld71dfr_kn_fq57str0000gn/T/ipykernel_12572/415546996.py:29: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seeds[i])


training seed 1 started


Training: 100%|██████████| 100/100 [00:24<00:00,  4.16epochs/s]


training seed 2 started


Training: 100%|██████████| 100/100 [00:23<00:00,  4.18epochs/s]


...

training seed 4 started


Training: 100%|██████████| 100/100 [00:27<00:00,  3.65epochs/s]


training seed 5 started


Training: 100%|██████████| 100/100 [00:27<00:00,  3.61epochs/s]

Training finished for all seeds.


In [17]:
result = []
models = ["SGD", "EI", "DP", "EO", "EODD"]
sol = [SGD, EI, DP, EO, EODD]
for i in range(len(models)):
    c = []
    c.append(models[i])
    res = sol[i]
    c.append(res['accuracy_mean'])
    c.append(res['accuracy_var'])
    c.append(res['ei_mean'])
    c.append(res['ei_var'])
    c.append(res['dp_mean'])
    c.append(res['dp_var'])
    c.append(res['eo_mean'])
    c.append(res['eo_var'])
    c.append(res['eodd_mean'])
    c.append(res['eodd_var'])
    result.append(c)

print(tabulate(result, headers=["model","accuracy_mean","accuracy_var","ei_mean","ei_var", "dp_mean", "dp_var","eo_mean","eo_var","eodd_mean","eodd_var"]))

model      accuracy_mean    accuracy_var     ei_mean      ei_var    dp_mean      dp_var    eo_mean      eo_var    eodd_mean    eodd_var
-------  ---------------  --------------  ----------  ----------  ---------  ----------  ---------  ----------  -----------  ----------
SGD             0.849456     0.000651137  0.0571562   0.00505918  0.125076   0.00898856  0.084543   0.00956585    0.084543   0.00956585
EI              0.832563     0.0069309    0.00885237  0.00450576  0.0468868  0.0194645   0.130263   0.0616879     0.130263   0.0616879
DP              0.817795     0.0317813    0.0422318   0.0227685   0.0303102  0.015677    0.131241   0.0659683     0.131241   0.0659683
EO              0.84579      0.00453176   0.0498401   0.0167915   0.110133   0.0229628   0.0248759  0.0145742     0.0428275  0.0171477
EODD            0.845843     0.00326828   0.0491      0.0100964   0.101968   0.012267    0.0137794  0.0109355     0.0360132  0.00754892
